In [1]:
from scipy.signal import butter, filtfilt, find_peaks, freqz, sosfilt
import glob
import os
from fnmatch import fnmatch
import h5py
import matplotlib as mpl
import matplotlib.pyplot as plt 
import numpy as np 
import pandas as pd 
import sys 
import multiprocessing
from multiprocessing import Pool

In [ ]:
getData(fileList)

In [2]:
def mkFileList(dataDir):

	#get list of files in dataDir, pack data*.h5 named files into fileList
	dataDirContents = os.listdir(dataDir)
	fileList = [file for file in dataDirContents if fnmatch(file, 'data*.h5')]
	fileList.sort()
	return fileList

def getData(aFile):
	dataDir = './data/'
	#init some parameters
	offData = np.zeros(2**23)
	onData = np.zeros(2**23)
	totalSets = 0
	totalAverages = 0

	#itterate through file list and open h5 files in 'r(ead)' mode
	print('ON FILE: ' + str(aFile))
	dataBin = h5py.File(dataDir + aFile, 'r')
	fileAverages = int(dataBin.attrs['averages'])
	allKeys = [aKey for aKey in dataBin.keys()]

	for aKey in allKeys[:]:
		print('ON KEY ' + str(aKey))
		dataset = pd.read_hdf(dataDir + aFile, key = aKey)

		offData = offData + np.asarray(dataset[dataset.keys()[0]])
		onData = onData + np.asarray(dataset[dataset.keys()[1]])
		totalSets += 1
		totalAverages += fileAverages
	offData = 10.*np.log10(2. * offData / totalSets  / (2**48 * 50. / 1000.))
	onData = 10.*np.log10(2. * onData / totalSets  / (2**48 * 50. / 1000.))


	return offData, onData, totalAverages

	
def getDataBen(dataDir):
	fileList = sorted(
		[aFile for aFile in glob.glob(dataDir + 'data*.h5')], 
		key=lambda x: int(x[x.index('data_') + 5: x.index('.h5')])
		)
	print(fileList)



	offData = np.zeros(2**23)
	onData = np.zeros(2**23)

	totalSets = 0
	totalAverages = 0
	for aFile in fileList[:100]:
		print('ON FILE: ' + str(aFile))
		dataBin = h5py.File(aFile, 'r')
		fileAverages = int(dataBin.attrs['averages'])
		allKeys = [aKey for aKey in dataBin.keys()]

		for aKey in allKeys[:]:
			print('ON KEY ' + str(aKey))
			dataset = pd.read_hdf(aFile, key = aKey)

			offData = offData + np.asarray(dataset[dataset.keys()[0]])
			onData = onData + np.asarray(dataset[dataset.keys()[1]])
			totalSets += 1
			totalAverages += fileAverages
	offData = 10.*np.log10(2. * offData / totalSets  / (2**48 * 50. / 1000.))
	onData = 10.*np.log10(2. * onData / totalSets  / (2**48 * 50. / 1000.))


	return offData, onData, totalAverages

def plotTermSigData(fileNameTerm, avgDataBicon, freqs):
	offDataTerm = np.zeros(2**23)
	onDataTerm = np.zeros(2**23)

	totalSets = 0
	print('ON FILE: ' + str(fileNameTerm))
	dataBin = h5py.File(fileNameTerm, 'r')
	allKeys = [aKey for aKey in dataBin.keys()]

	for aKey in allKeys[:]:
		print('ON KEY ' + str(aKey))
		dataset = pd.read_hdf(fileNameTerm, key = aKey)

		offDataTerm = offDataTerm + np.asarray(dataset[dataset.keys()[0]])
		onDataTerm = onDataTerm + np.asarray(dataset[dataset.keys()[1]])
		totalSets += 1

	offDataTerm = 10.*np.log10(2. * offDataTerm / totalSets  / (2**48 * 50. / 1000.))
	onDataTerm = 10.*np.log10(2. * onDataTerm / totalSets  / (2**48 * 50. / 1000.))
	peakIndices = find_peaks(avgDataBicon[1:] - onDataTerm[1:], threshold = 1.75)[0]
	peakVals = np.asarray([(avgDataBicon[1:] - onDataTerm[1:])[x] for x in peakIndices]) 

	plt.xlabel('Frequency (MHz)', labelpad = 15, **label_font)
	plt.ylabel('Ratio (dB)', **label_font)
	#plt.plot(freqs[1:], onDataTerm[1:], alpha = 0.7, label = 'Bicon Sig Gen Off')
	plt.plot(avgDataBicon[1:] - onDataTerm[1:], alpha = 0.7, label = 'Bicon Sig Gen On')
	plt.plot(peakIndices, peakVals, 'r*')

	plt.title('Ratio Bicon, Injected 123.456MHz vs No Injection')
	plt.show()
	#plt.legend(prop = legend_font)
	return peakIndices

In [5]:
dataDir = './data/'
fileList = mkFileList(dataDir)
#print(fileList)

if __name__ == '__main__':
    pool = Pool(processes=8)
    pool.map(getData, fileList[0:8])

ON FILE: data_0.h5ON FILE: data_109.h5

ON KEY measdata_0
ON KEY measdata_1744
ON KEY measdata_1
ON KEY measdata_1745
ON KEY measdata_10
ON KEY measdata_1746
ON KEY measdata_11
ON KEY measdata_1747
ON KEY measdata_12
ON KEY measdata_1748
ON KEY measdata_13ON KEY measdata_1749

ON KEY measdata_1750
ON KEY measdata_14
ON KEY measdata_1751
ON KEY measdata_15
ON KEY measdata_1752ON KEY measdata_2

ON KEY measdata_3
ON KEY measdata_1753
ON KEY measdata_4
ON KEY measdata_1754
ON KEY measdata_5
ON KEY measdata_1755
ON KEY measdata_1756
ON KEY measdata_6
ON KEY measdata_1757
ON KEY measdata_7
ON KEY measdata_1758
ON KEY measdata_8
ON KEY measdata_1759
ON KEY measdata_9
ON FILE: data_119.h5
ON KEY measdata_1904
ON FILE: data_122.h5
ON KEY measdata_1952
ON KEY measdata_1905
ON KEY measdata_1953
ON KEY measdata_1906
ON KEY measdata_1954
ON KEY measdata_1907
ON KEY measdata_1955
ON KEY measdata_1908
ON KEY measdata_1956
ON KEY measdata_1909
ON KEY measdata_1957
ON KEY measdata_1910
ON KEY measdata

In [ ]:
dataDir = './data/'
getData(dataDir)

In [ ]:
termData